|          | GPT3.5 | GPT4 |
|----------|----------|----------|
| GPT3.5   | 1      | 0      |
| GPT4     | 1      | 1      |

In [ ]:
from langchain.chat_models import AzureChatOpenAI

# assuming LLM api keys have been set in the environment

llm_35 = AzureChatOpenAI(
    deployment_name="gpt-35-turbo",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm_4 = AzureChatOpenAI(
    deployment_name="gpt-4-32k",
    temperature=0,
    request_timeout=20,
    max_retries=1,
    client=None,
)

llm_4.predict('What American cartoonist is the creator of Andy Lippincott?')

In [ ]:
from langchain.prompts.prompt import PromptTemplate

_SIMPLE_PROMPT = """{context}"""


OTHER_PROMPT = PromptTemplate(
    input_variables=["context"],
    template=_SIMPLE_PROMPT,
)

In [ ]:
from graph_feedback_chain import GraphFeedbackChain

graph_chain = GraphFeedbackChain.from_llms({"gpt-35-turbo": llm_35, "gpt-4": llm_4}, prompt=OTHER_PROMPT)


In [ ]:
inputs={"selected": "gpt-35-turbo", "context":"What American cartoonise is the creator of Andy Lippincott?"}

graph_chain.run(inputs)

In [ ]:
from response_validator import *

template = "PLEASE RESPOND ONLY WITH A SIGNLE FLOAT AND NO OTHER TEXT EXPLANATION\n You are a VERY VERY strict judge that is called on to rank a response based on given criteria.\
                You must respond with your ranking by providing a single float within the range [-1, 1], -1 being very bad response and 1 being very good response."
system_message_prompt = SystemMessagePromptTemplate.from_template(template)
human_template = "Given the answer: {answer} to the question: {context}, how would you rate the answer of {llm_response}"
human_message_prompt = HumanMessagePromptTemplate.from_template(
    human_template
)

REWARD_PROMPT = ChatPromptTemplate.from_messages(
    [system_message_prompt, human_message_prompt]
)

In [ ]:
from vw_example_builder import GraphFeedbackTextEmbedder
import rl_chain

chain = rl_chain.PickBest.from_chain(
    llm_chain=graph_chain,
    model_save_dir="./gf_models_actual_gf",  # where to save the model checkpoints
    response_validator = AutoValidatePickOne(llm=llm_35, prompt=REWARD_PROMPT),
    vw_cmd=['--cb_explore_adf', '--quiet', '--interactions=::', '--coin', '--graph_feedback'],
    prompt=OTHER_PROMPT,
    text_embedder=GraphFeedbackTextEmbedder('bert-base-nli-mean-tokens'),
)

In [ ]:

inputs = {"actions":["gpt-35-turbo", "gpt-4"], "graph": "0,0,1 0,1,0 1,0,1 1,1,1"}

for q, a in [("which country is john berry from", "United States of America country in North America")]:
    try:
        inputs["context"] = q
        inputs["answer"] = a
        r = chain.run(inputs)
        resp = r["response"]
        chosen_action = r["response_result"].chosen_action
        cost = r["response_result"].cost
        print(f"m: {chosen_action} -- answer: {a} -- response: {resp}")
        print(r["response_result"].cost)
        print(r["response_result"].chosen_action)

    except Exception as e:
        print(e)
